In [0]:
# Utilities
from tqdm.auto import tqdm
tqdm.pandas()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# word cleaning
import string
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS

# Models
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

PATH = "/content/drive/My Drive/sen_ana/"

In [20]:
df = pd.read_csv(PATH+"candidate_hospitals_translated.csv")
df

,comment,score,hospital,en
0,เคยได้ไปลองใช้บริการคลีนิคนอกเวลาของรามาเมื่อต...,5,ramathibodi-hospital,"Ever were going to try to use the service, the..."
1,ผมเคยไปทำเลสิกที่นี้ ดีมากครับตอนนี้ผ่ามาจะ2ปี...,4,ramathibodi-hospital,I've been to Lasik the this good now. coming t...
2,รักษามาหลายปีแล้วตั้งแต่อายุ14 ตอนนี้จะจบปริญญ...,5,ramathibodi-hospital,Treatment for years since the age of 14 will n...
3,ที่ตึกพระเทพชั้น 2 สถานที่เครื่องมือดูทันสมัย ...,3,ramathibodi-hospital,"Building the floor 2. place tools, modern look..."
4,Its a public hospital so service is bad (nurse...,4,ramathibodi-hospital,Its a public hospital so service is bad (the n...
...,...,...,...,...
702,ค่ำวันที่ 26 ตุลาคม 2559 เหตุการณ์ในวันนั้นฉัน...,5,king-chulalongkorn-memorial-hospital,"Evening of October 26, 2559 event on that day,..."
703,ทำเพื่ออะไรรีวิวนี้แต่เราไปรักษารพจุฬาก็ได้รับ...,5,king-chulalongkorn-memorial-hospital,"Do what for this review, but we to maintain th..."
704,ทาง รพ. บริการคนไข้ดีมาก ใส่ใจผู้ป่วย เป็นกันเ...,5,king-chulalongkorn-memorial-hospital,"The hospital. Services, patient, very attentiv..."
705,หมอเก่ง,3,king-chulalongkorn-memorial-hospital,The doctor brilliantly


In [0]:
# Stopword removal, Lemmanization and punctuation removal

nlp = spacy.load('en_core_web_sm')
punct = string.punctuation
num =  string.digits

def stop(text):
  cleaned_token = []
  doc = nlp(text) 

  for word in doc:
    if word.lemma_ not in STOP_WORDS and word.lemma_ not in punct:
      cleaned_token.append(word.lemma_)
  return cleaned_token

In [22]:
word_clean = pd.DataFrame(df.en.progress_apply(lambda x: stop(x)))
word_clean = pd.DataFrame(word_clean.en.progress_apply(lambda x:' '.join(x)))
word_clean['score'] = df.score
word_clean['hospital'] = df.hospital

In [42]:
# score 3 ignored, score 1,2 as bad, score 4,5 as good

word_clean = word_clean.loc[word_clean.score != 3]
word_clean["sen"] = 0
word_clean.loc[word_clean.score.isin([4,5]),["sen"]] = 1
word_clean

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,en,score,hospital,sen
0,try use service clinic Rama beginning treatmen...,5,ramathibodi-hospital,1
1,-PRON- Lasik good come year price good middle ...,4,ramathibodi-hospital,1
2,treatment year age 14 finish -PRON- degree sle...,5,ramathibodi-hospital,1
4,-PRON- public hospital service bad nurse compl...,4,ramathibodi-hospital,1
5,-PRON- friend disease pulmonary leak doctor ca...,5,ramathibodi-hospital,1
...,...,...,...,...
701,large hospital lot people doctor nurse good se...,5,king-chulalongkorn-memorial-hospital,1
702,evening October 26 2559 event day -PRON- remem...,5,king-chulalongkorn-memorial-hospital,1
703,review -PRON- maintain CHULA treat today .... ...,5,king-chulalongkorn-memorial-hospital,1
704,hospital service patient attentive patient fri...,5,king-chulalongkorn-memorial-hospital,1


In [0]:
word_clean.to_csv(PATH+"word_clean.csv",index=False)

# Model Selection

In [0]:
model = [MultinomialNB(),BernoulliNB(),LogisticRegression(),SGDClassifier(),SVC(),KNeighborsClassifier(),MLPClassifier()]

In [0]:
X = word_clean["en"]
y = word_clean["sen"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
def select_model(model):
  for i in model:    
    clf = Pipeline([
      ("tfidf", TfidfVectorizer(tokenizer=None , stop_words=None,ngram_range=(1,1), use_idf=True)),
      ("clf", i)])
    # cross validation 10 folds on training set only, test set will be used after hyperparameter tuning
    scores = cross_val_score(clf, X_train, y_train, cv=10)
    print(f"\n Model {str(i)} got accuracy %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [56]:
select_model(model)


 Model MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) got accuracy 0.80 (+/- 0.00)

 Model BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) got accuracy 0.82 (+/- 0.05)

 Model LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) got accuracy 0.81 (+/- 0.02)

 Model SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False) g

In [0]:
# MLP and SGD perform the best
# KNN perform second best

# Multi-layer Perceptron
### Hyperparamter Tuning

In [0]:
clf_mlp = Pipeline([
      ("tfidf", TfidfVectorizer()),
      ("clf", MLPClassifier(max_iter=100))
])

"""parameters = {"tfidf__ngram_range": [(1, 1), (1, 2)],
              "tfidf__use_idf": (True, False),
              "clf__activation": ("identity", "logistic", "tanh", "relu"),
              "clf__solver":("lbfgs", "sgd", "adam"),
              "clf__learning_rate":("constant", "invscaling", "adaptive")}"""

# the tuned parameters are reduced due to runtime constraints
parameters = {"tfidf__ngram_range": [(1, 1), (1, 2)],
              "tfidf__use_idf": (True, False),
              "clf__activation": ("tanh", "relu"),
              "clf__solver":("lbfgs","sgd", "adam"),
              "clf__learning_rate":("constant", "adaptive")}

gs_clf = GridSearchCV(clf_mlp, parameters, cv=3, n_jobs=-1)

In [107]:
gs_clf.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [108]:
for param_name in sorted(parameters.keys()):
  print("Best parameters: \n %s: %r" % (param_name, gs_clf.best_params_[param_name]))

Best parameters: 
 clf__activation: 'relu'
Best parameters: 
 clf__learning_rate: 'adaptive'
Best parameters: 
 clf__solver: 'lbfgs'
Best parameters: 
 tfidf__ngram_range: (1, 2)
Best parameters: 
 tfidf__use_idf: True


In [111]:
# tuned
clf_mlp = Pipeline([
      ("tfidf", TfidfVectorizer(ngram_range=(1,2),use_idf=True)),
      ("clf", MLPClassifier(activation="relu",learning_rate="adaptive",solver="lbfgs", max_iter=100))
])
clf_mlp.fit(X_train,y_train)
y_pred = clf_mlp.predict(X_test)
accuracy_score(y_test,y_pred)

0.8372093023255814

In [112]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.44      0.53        27
           1       0.86      0.94      0.90       102

    accuracy                           0.84       129
   macro avg       0.77      0.69      0.72       129
weighted avg       0.82      0.84      0.82       129



# SGD
### Hyperparameter Tuning

In [0]:
clf_sgd = Pipeline([
      ("tfidf", TfidfVectorizer()),
      ("clf", SGDClassifier())
])
parameters = {"tfidf__ngram_range": [(1, 1), (1, 2), (1,3)],
              "tfidf__use_idf": [True, False],
              "clf__alpha": [1e-3,1e-4,1e-5],
              "clf__penalty":['none','l2','l1']}
gs_clf = GridSearchCV(clf_sgd, parameters, cv=10, n_jobs=-1)

In [91]:
gs_clf.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        

In [92]:
for param_name in sorted(parameters.keys()):
  print("Best parameters: \n %s: %r" % (param_name, gs_clf.best_params_[param_name]))

Best parameters: 
 clf__alpha: 0.0001
Best parameters: 
 clf__penalty: 'l2'
Best parameters: 
 tfidf__ngram_range: (1, 3)
Best parameters: 
 tfidf__use_idf: False


In [95]:
# tuned
clf_sgd = Pipeline([
      ("tfidf", TfidfVectorizer(ngram_range=(1,3),use_idf=False)),
      ("clf", SGDClassifier(alpha=0.0001, penalty="l2"))
])
clf_sgd.fit(X_train,y_train)
y_pred = clf_sgd.predict(X_test)
accuracy_score(y_test,y_pred)

0.8294573643410853

In [96]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.52      0.56        27
           1       0.88      0.91      0.89       102

    accuracy                           0.83       129
   macro avg       0.74      0.72      0.73       129
weighted avg       0.82      0.83      0.82       129



# K Nearest Neighbours
### Hyperparameter Tuning

In [0]:
clf_knn = Pipeline([
      ("tfidf", TfidfVectorizer()),
      ("clf", KNeighborsClassifier())
])
parameters = {"tfidf__ngram_range": [(1, 1), (1, 2), (1,3)],
              "tfidf__use_idf": [True, False],
              "clf__n_neighbors": [5,10,20],
              "clf__weights":["uniform","distance"],
              "clf__algorithm":["auto","ball_tree","kd_tree","brute"],
              #1 for manhattan, 2 for euclidean
              "clf__p": [1,2]}

gs_clf = GridSearchCV(clf_knn, parameters, cv=3, n_jobs=-1)

In [86]:
gs_clf.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [87]:
for param_name in sorted(parameters.keys()):
  print("Best parameters: \n %s: %r" % (param_name, gs_clf.best_params_[param_name]))

Best parameters: 
 clf__algorithm: 'auto'
Best parameters: 
 clf__n_neighbors: 5
Best parameters: 
 clf__p: 2
Best parameters: 
 clf__weights: 'uniform'
Best parameters: 
 tfidf__ngram_range: (1, 2)
Best parameters: 
 tfidf__use_idf: True


In [97]:
# tuned
clf_knn = Pipeline([
      ("tfidf", TfidfVectorizer(ngram_range=(1,2),use_idf=True)),
      ("clf", KNeighborsClassifier(algorithm="auto", n_neighbors=5, p=2, weights="uniform"))
])
clf_knn.fit(X_train,y_train)
y_pred = clf_knn.predict(X_test)
accuracy_score(y_test,y_pred)

0.7984496124031008

In [98]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      0.15      0.24        27
           1       0.81      0.97      0.88       102

    accuracy                           0.80       129
   macro avg       0.69      0.56      0.56       129
weighted avg       0.76      0.80      0.75       129



# Review

Model | F1 Score for Positive | F1 Score for Negative | Accuracy
-|-|-|-
Multi-layer Perceptron | 0.86 | 0.67 | 0.84
Stochastic Gradient Descent | 0.88 | 0.61 | 0.83
K Nearest Neighbours | 0.81 | 0.57 | 0.80
Sentiment Intensity Analyzer | 0.86 | 0.49 | 0.80

In [82]:
word_clean.groupby("sen").size()

sen
0    127
1    514
dtype: int64

Probably because my dataset has significantly less samples for negative sentiment. <br>
The models performs significantly worse for negative sentiments classification. <br>
In conclusion, **Multi-layer Perceptron** performs the best overall, out of the three candidates. <br>
On the other hand, all three models performs better or as accurate as the **Sentiment Intensity Analyzer**.